In [1]:
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.InteractiveSession(config=config)

In [2]:
# Importing data
dn_input = np.load('/home/beto/Documents/projects/DoggoNet/DN-dataset.npz')

x_train = dn_input['X_train']
x_test = dn_input['X_test']
y_train = dn_input['y_train']
y_test = dn_input['y_test']

print('Training set shape:', x_train.shape)
print('Testing set shape:', x_test.shape)

# convert class
num_classes = 4
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Model initialization
model = keras.Sequential([keras.Input(shape=(225, 300, 3))])


# Defining building blocks
def add_conv_lyr(num_neurons, k_size, act_func, times=1):
    for i in range(times):
        model.add(layers.Conv2D(num_neurons, kernel_size=k_size, activation=act_func))


def add_max_pool_lyr(p_size, times=1):
    for i in range(times):
        model.add(layers.MaxPooling2D(pool_size=p_size))


def add_dense_lyr(num_neurons, act_func, times=1):
    for i in range(times):
        model.add(layers.Dense(num_neurons, activation=act_func))


# Adding logic to the model
add_conv_lyr(64, (3, 3), 'relu')
add_max_pool_lyr((2, 2))
add_conv_lyr(48, (3, 3), 'relu')
add_max_pool_lyr((2, 2))
add_conv_lyr(32, (3, 3), 'relu')
add_max_pool_lyr((2, 2))

add_dense_lyr(100, 'tanh', 3)

# Ending model
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
add_dense_lyr(num_classes, 'softmax')
model.summary()

batch_size = 34 * 3
epochs = 12

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print('\n-------------------- MODEL COMPILED AND READY --------------------\n\n')

Training set shape: (3400, 225, 300, 3)
Testing set shape: (600, 225, 300, 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 223, 298, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 149, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 147, 48)      27696     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 73, 48)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 71, 32)        13856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 35, 32)        0         
____________________________________________

In [3]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/12
30/30 [==============================] - 168s 6s/step - loss: 1.3130 - accuracy: 0.4167 - val_loss: 0.9478 - val_accuracy: 0.5971
Epoch 2/12
30/30 [==============================] - 178s 6s/step - loss: 0.5846 - accuracy: 0.7647 - val_loss: 0.3445 - val_accuracy: 0.8735
Epoch 3/12
30/30 [==============================] - 181s 6s/step - loss: 0.2640 - accuracy: 0.9118 - val_loss: 0.1999 - val_accuracy: 0.9324
Epoch 4/12
30/30 [==============================] - 183s 6s/step - loss: 0.1661 - accuracy: 0.9431 - val_loss: 0.1830 - val_accuracy: 0.9265
Epoch 5/12
30/30 [==============================] - 185s 6s/step - loss: 0.1497 - accuracy: 0.9542 - val_loss: 0.2516 - val_accuracy: 0.9088
Epoch 6/12
30/30 [==============================] - 185s 6s/step - loss: 0.1027 - accuracy: 0.9660 - val_loss: 0.1064 - val_accuracy: 0.9618
Epoch 7/12
30/30 [==============================] - 187s 6s/step - loss: 0.0486 - accuracy: 0.9866 - val_loss: 0.1504 - val_accuracy: 0.9412
Epoch 8/12
30

In [ ]:
#keras.backend.clear_session()

In [55]:
# Testing trained model
def pick_image(num, dataset):
    d = {0:'Jack',1:'Luna',2:'Volt',3:'Katy'}
    if dataset == 'train':
        etiqueta = dn_input['y_train'][num]
        foto = dn_input['X_train'][num]
    elif dataset == 'test':
        etiqueta = dn_input['y_test'][num]
        foto = dn_input['X_test'][num]
    foto = np.multiply(foto, 255).astype(np.int8)
    print('This is: {}\nShowing image from {}ing set'.format(d[etiqueta[0]], dataset))
    Image.fromarray(foto, 'RGB').show()

def test_image(num):
    d = {0:'Jack',1:'Luna',2:'Volt',3:'Katy'}
    etiqueta = dn_input['y_test'][num]
    foto = dn_input['X_test'][num]
    foto = np.multiply(foto, 255).astype(np.int8)
    result = model.predict(dn_input['X_test'][num].reshape(1,225,300,3)).flatten()
    val = result.max()
    i = np.where(np.isclose(result, val))[0][0]
    print('Neural network predicts this is {}\nShnp.where(np.isclose(a, value))owing image to compare'.format(d[i]))
    Image.fromarray(foto, 'RGB').show()

In [72]:
test_image(15) # select from 0 to 599

Neural network predicts this is Volt
Showing image to compare


In [76]:
model.save(
    'model_24feb21_9706_h5',
    overwrite=True,
    include_optimizer=True,
    save_format='h5',
    signatures=None,
    options=None
)